In [21]:
from data import Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers

In [36]:
data = Data(from_date=datetime(2022, 1, 1), to_date=datetime(2022, 2, 15))
dataset = data.normalize_data(depth=40, foreseeing=10, column_titel="close")

X_DATA = []
Y_DATA = []
for i in dataset:
    X_DATA.append(i[0])
    Y_DATA.append([i[1][3]])

data_size = len(X_DATA)
train_data = (np.array(X_DATA[0:int(data_size*0.8)]), np.array(Y_DATA[0:int(data_size*0.8)]))
valid_data = (np.array(X_DATA[int(data_size*0.8)::]), np.array(Y_DATA[int(data_size*0.8)::]))

train_data_reshaped = np.expand_dims(train_data[0], axis=1) 
train_data_reshaped = np.tile(train_data_reshaped, (1, train_data[0].shape[0], 1))

# valid_data_reshaped = np.expand_dims(valid_data[0], axis=1) 
# valid_data_reshaped = np.tile(valid_data_reshaped, (1, valid_data[0].shape[0], 1))
train_data[0].shape

(555, 40)

In [37]:
# class ResetStatesCallback(keras.callbacks.Callback):
#   def on_epoch_begin(self, epoch, logs):
#     self.model.reset_states()

In [38]:
model = keras.models.Sequential([
  keras.layers.LSTM(40, return_sequences=True, stateful=True, batch_input_shape=(None, 555, 40)),
  keras.layers.LSTM(100, return_sequences=True, stateful=True),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200.0)
])
optimizer = keras.optimizers.SGD(learning_rate=5e-7, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
# reset_states = ResetStatesCallback()
# model_checkpoint = keras.callbacks.ModelCheckpoint(
#     "my_checkpoint.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=50)
model.fit(train_data_reshaped, train_data[1], epochs=500)

ValueError: If a RNN is stateful, it needs to know its batch size. Specify the batch size of your input tensors: 
- If using a Sequential model, specify the batch size by passing a `batch_input_shape` argument to your first layer.
- If using the functional API, specify the batch size by passing a `batch_shape` argument to your Input layer.